## Classify Text into Categories using Natural Language API

### Overview: 
The Cloud Natural Language API lets you extract entities from text, perform sentiment and syntactic analysis, and classify text into categories. In this lab, we'll focus on text classification. Using a database of 700+ categories, this API feature makes it easy to classify a large dataset of text.

#### Objectives:

- Creatie a Natural Language API request and calling the API with curl
- Use the NL API's text classification feature
- Use text classification to understand a dataset of news articles

1.) Generate and save API key. Using Cloud Shell:

```
export API_KEY=<YOUR_API_KEY>
```

2.) Classify a news article

- Using the Natural Language API's classifyText method, you can sort text data into categories with a single API call. This method returns a list of content categories that apply to a text document. These categories range in specificity, from broad categories like /Computers & Electronics to highly specific categories such as /Computers & Electronics/Programming/Java (Programming Language). A full list of 700+ possible categories can be found here.

- We'll start by classifying a single article, and then we'll see how we can use this method to make sense of a large news corpus. To start, let's take this headline and description from a New York Times article in the food section:

```"A Smoky Lobster Salad With a Tapa Twist. This spin on the Spanish pulpo a la gallega skips the octopus, but keeps the sea salt, olive oil, pimentón and boiled potatoes."```

- In your Cloud Shell environment, create a request.json file with the code below. You can either create the file using one of your preferred command line editors (nano, vim, emacs) or use the Cloud Shell code editor (click the pencil button):

```
{
  "document":{
    "type":"PLAIN_TEXT",
    "content":"A Smoky Lobster Salad With a Tapa Twist. This spin on the Spanish pulpo a la gallega skips the octopus, but keeps the sea salt, olive oil, pimentón and boiled potatoes."
  }
}
```
- Save
```
curl "https://language.googleapis.com/v1/documents:classifyText?key=${API_KEY}" \
  -s -X POST -H "Content-Type: application/json" --data-binary @request.json
```


3.) Classifying a large text dataset

- To see how the classifyText method can help us understand a dataset with lots of text, you'll use this public dataset of BBC news articles. The dataset consists of 2,225 articles in five topic areas (business, entertainment, politics, sports, tech) from 2004 - 2005. A subset of these articles are in a public Google Cloud Storage bucket. Each of the articles is in a .txt file.

- To examine the data and send it to the Natural Language API, you'll write a Python script to read each text file from Cloud Storage, send it to the classifyText endpoint, and store the results in a BigQuery table. BigQuery is Google Cloud's big data warehouse tool - it lets you easily store and analyze large data sets.

- To see the type of text you'll be working with, run the following command to view one article (gsutil provides a command line interface for Cloud Storage):

```
gsutil cat gs://text-classification-codelab/bbc_dataset/entertainment/001.txt
```

4.) Creating a BigQuery table for our categorized text data

- Before sending the text to the Natural Language API, you need a place to store the text and category for each article.
- Navigate to the BigQuery in the Console. You will need to log in - remember to use your Qwiklabs credentials
- Select Create new datase and name dataset: news_classification_dataset. Leave the defaults in the Data location and Data expiration fields.
- Click on the dropdown arrow next to your dataset name and select Create new table. Under Source Data, select "Create empty table". Then name your table article_data and add the following 3 fields in the schema: article_text, category, and confidence.

5.) Classifying news data and storing the result in BigQuery

- Before writing a script to send the news data to the Natural Language API, you need to create a service account. This will be used to authenticate to the Natural Language API and BigQuery from a Python script.
- First, back in Cloud Shell, export the name of your Cloud project as an environment variable. Replace <your_project_name> with the GCP Project ID found in the CONNECTION DETAILS section of the lab:

```
export PROJECT=<your_project_name>

# from cloud shell
gcloud iam service-accounts create my-account --display-name my-account
gcloud projects add-iam-policy-binding $PROJECT --member=serviceAccount:my-account@$PROJECT.iam.gserviceaccount.com --role=roles/bigquery.admin
gcloud iam service-accounts keys create key.json --iam-account=my-account@$PROJECT.iam.gserviceaccount.com
export GOOGLE_APPLICATION_CREDENTIALS=key.json
```

- Now you're ready to send the text data to the Natural Language API!
- To do that, write a Python script using the Python module for Google Cloud. You can accomplish the same thing from any language, there are many different cloud client libraries.
- Create a file called classify-text.py and copy the following into it. Replace YOUR_PROJECT with your GCP Project ID.

```
from google.cloud import storage, language, bigquery

# Set up our GCS, NL, and BigQuery clients
storage_client = storage.Client()
nl_client = language.LanguageServiceClient()
# TODO: replace YOUR_PROJECT with your project id below
bq_client = bigquery.Client(project='YOUR_PROJECT')

dataset_ref = bq_client.dataset('news_classification_dataset')
dataset = bigquery.Dataset(dataset_ref)
table_ref = dataset.table('article_data') # Update this if you used a different table name
table = bq_client.get_table(table_ref)

# Send article text to the NL API's classifyText method
def classify_text(article):
        response = nl_client.classify_text(
                document=language.types.Document(
                        content=article,
                        type=language.enums.Document.Type.PLAIN_TEXT
                )
        )
        return response

rows_for_bq = []
files = storage_client.bucket('text-classification-codelab').list_blobs()
print("Got article files from GCS, sending them to the NL API (this will take ~2 minutes)...")

# Send files to the NL API and save the result to send to BigQuery
for file in files:
        if file.name.endswith('txt'):
                article_text = file.download_as_string()
                nl_response = classify_text(article_text)
                if len(nl_response.categories) > 0:
                        rows_for_bq.append((article_text, nl_response.categories[0].name, nl_response.categories[0].confidence))

print("Writing NL API article data to BigQuery...")
# Write article text + category data to BQ
errors = bq_client.insert_rows(table, rows_for_bq)
assert errors == []
```
- Run:

```
python classify-text.py

```

#### We're using the google-cloud Python client library to access Cloud Storage, the Natural Language API, and BigQuery. First, a client is created for each service; then references are created to the BigQuery table. files is a reference to each of the BBC dataset files in the public bucket. We iterate through these files, download the articles as strings, and send each one to the Natural Language API in our classify_text function. For all articles where the Natural Language API returns a category, the article and its category data are saved to a rows_for_bq list. When classifying each article is done, the data is inserted into BigQuery using insert_rows().

- Navigate to the article_data table in the BigQuery tab and click Query Table. Edit the results in the New Query box so it looks like this, replacing the brackets with a backtick (on the tilde key, next to the 1) not a single quote, adding an asterisk between SELECT and FROM, and replacing the : after your project name with a ‘ .

```
SELECT * FROM YOUR_PROJECT.news_classification_dataset.article_data

```

- Click the Show Options button and uncheck the Use Legacy SQL box.
- Run Query.
- Review data when the query completes. Scroll to the right to see the category column.

6.) Analyzing categorized news data in BigQuery
 
 - In Big Query: 
 
```
SELECT
     category,
      COUNT(*) c
FROM
    `YOUR_PROJECT.news_classification_dataset.article_data`
GROUP BY
    category
ORDER BY
    c DESC   
```

- To find additional categories:

```
SELECT * FROM `YOUR_PROJECT.news_classification_dataset.article_data`
WHERE category = "/Arts & Entertainment/Music & Audio/Classical Music"
```

- High confidence only:

```
SELECT
  article_text,
  category
FROM `YOUR_PROJECT.news_classification_dataset.article_data`
WHERE cast(confidence as float64) > 0.9
```